In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

import numpy as np
import cv2
import os
import imutils

from sklearn.metrics import classification_report 
from sklearn.metrics import f1_score ,accuracy_score

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

from os import listdir
from os.path import isfile, join
import tensorflow
import time

In [2]:
batch_size = 32
num_classes = 4
#epochs = 16
data_augmentation = True
num_predictions = 20
width = 150
height = 150
#images = np.empty((0, 150,150,3))
labels = []
rawImage = []

In [3]:
#model 10 ตอนเทรน ภาพครบ  epoc 13
model10 = Sequential()

model10.add(Conv2D(32, (3, 3), padding='same', input_shape=(150, 150, 3)))
model10.add(Activation('relu'))
model10.add(MaxPooling2D(pool_size=(2, 2)))

model10.add(Conv2D(64, (3, 3), padding='same'))
model10.add(Activation('relu'))
model10.add(MaxPooling2D(pool_size=(2, 2)))

model10.add(Conv2D(128, (3, 3), padding='same'))
model10.add(Activation('relu'))
model10.add(MaxPooling2D(pool_size=(2, 2)))

model10.add(Flatten())
model10.add(Dense(128))
model10.add(Activation('relu'))
model10.add(Dropout(0.5))
model10.add(Dense(num_classes))
model10.add(Activation('softmax'))
opt = keras.optimizers.Adam(lr=0.0001, decay=1e-6)
model10.compile(loss="binary_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])
model10.load_weights("/content/drive/My Drive/korat_sauce/train_sauce10.h5")

In [4]:
cap = cv2.VideoCapture('/content/drive/My Drive/korat_sauce/VID_20200913_161602.3gp')
sauce_count = 0
list_white_area =[]
i=0
check_status = 0
pic = 0
balck_count =0
maeyupa = 0
ocean = 0
zapnuer = 0
zapnuerchick = 0 
try:
    while(cap.isOpened()):
        # Capture frame-by-frame
        ret, frame = cap.read()

        # Our operations on the frame come here
        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        #(thresh, blackAndWhiteImage) = cv2.threshold(gray, 90, 255, cv2.THRESH_BINARY)
        ksize = (40, 40)  
        img = cv2.blur(frame, ksize)  
        hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
        lower_white = np.array([0,0,180])
        upper_white = np.array([255,255,255])
        blackAndWhiteImage = cv2.inRange(hsv, lower_white, upper_white)
        # Display the resulting frame
        #cv2.imshow('frame',frame)
        #cv2.imwrite('pic//all.'+str(i)+'.jpg',frame)
        if(i%1==0):
            #cv2.imshow('frame',frame[250:500,550:840])
            #cv2.imshow('blur',img[250:500,550:840])
            #cv2.imshow('Gray image', gray)
            #cv2.imshow('Black white image', blackAndWhiteImage[250:500,550:840])
            images_array = np.array(blackAndWhiteImage)
            images_array = images_array.astype('float32')
            images_array /=255
            white_area = 0
            for j in range(250,500):
                white_area+=sum(images_array[j,550:840])
            list_white_area.append(white_area)
            if(white_area>=50750):
                if(check_status==0):
                    if(balck_count>3):
                        sauce_count+=1
                        check_status = 1
                        #cv2.imwrite('count_pic//all.'+str(pic)+'.jpg',frame)
                        #cv2.imwrite('count_pic//all.'+str(pic)+'.black.jpg',blackAndWhiteImage)
                        image = cv2.resize(frame, (width,height))
                        img_list = []
                        img_list.append(image)
                        img_list = np.array(img_list)
                        img_list = img_list.astype('float32')
                        img_list /= 255.
                        prediction = model10.predict(np.array([img_list[0]]))
                        y_classes = prediction.argmax(axis=-1)
                        if(y_classes[0]==0):
                          maeyupa+=1
                        elif(y_classes[0]==1):
                            ocean+=1
                        elif(y_classes[0]==2):
                            zapnuer+=1
                        elif(y_classes[0]==3):
                            zapnuerchick+=1

                        pic+=1
                        balck_count = 0
            else:
                balck_count+=1
                if(check_status==1):
                    check_status = 0
                
            
            
            #print(len(blackAndWhiteImage))
            #print(len(blackAndWhiteImage[0]))
            #time.sleep(0.3)
            #break
        i=i+1
        if(i==25):
            i = 0
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except:
    pass
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
print("number of pic = ",pic)
print("maeyupa = ",maeyupa)
print("ocean = ",ocean)
print("zapnuer = ",zapnuer)
print("zapnuerchick = ",zapnuerchick)

number of pic =  99
maeyupa =  23
ocean =  23
zapnuer =  22
zapnuerchick =  31
